In [1]:
# script to combine summary urban form stats for all cities
# last edit Aug 5 2023 Peter Berrill

# load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from shapely.wkt import loads,dumps
from shapely import speedups
from shapely.geometry import Point, LineString, Polygon
speedups.enable()
from pyproj import CRS
from pysal.lib import weights
import pickle
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from citymob import import_csv_w_wkt_to_gdf
import networkx as nx
import osmnx as ox

crs0=3035

cities=['Berlin','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam','Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Paris','Toulouse','Madrid','Wien']
countries=['Germany','Germany','Germany','Germany','Germany','Germany','Germany','Germany','France','France','France','France','France','France','France','France','France','Spain','Austria']
ua_year=['2018','2018','2018','2018','2018','2018','2018','2018','2012','2018','2018','2018','2012','2018','2018','2012','2012','2018','2012']
ua_ver=['v013','v013','v013','v013','v013','v013','v013','v013','revised_v021','v013','v013','v013','revised_v021','v013','v013','revised_v021','revised_v021','v013','revised_v021']

metrics=['n','m','k_avg','intersection_density_km','clean_intersection_density_km','street_length_total','street_density_km','streets_per_node_avg','street_length_avg']
nw_type='drive'

In [2]:
bld_dens_city=[]
pop_dens_city=[]
pop_city=[]
area_city=[]
d2c=[]
d2sc=[]
urb_fab=[]
comm=[]
int_dens=[]
street_len=[]
bike_share=[]



In [3]:
def get_UF_avg(city):
    print(city)
    country=countries[cities.index(city)]
    if city == 'Frankfurt am Main': 
        fp='../../MSCA_data/BuildingsDatabase/clips/eubucco_frankfurt_am_main.shp'
    else:
        fp='../../MSCA_data/BuildingsDatabase/clips/eubucco_' + city + '.shp'
    buildings_gdf=gpd.read_file(fp)

    # read in file of city boundaries
    if country=='Germany':
        fp='../outputs/city_boundaries/' + city + '_basic.csv'
    else: 
        fp='../outputs/city_boundaries/' + city + '.csv'
    gdf_boundary = import_csv_w_wkt_to_gdf(fp,crs=crs0,geometry_col='geometry')

    buildings_gdf=gpd.sjoin(buildings_gdf,gdf_boundary)

    # calculate the area of the buidlings from the database
    buildings_gdf["area"] = buildings_gdf["geometry"].area
    # calculate the centerpoint of each building geometry, 
    buildings_gdf["center"] = buildings_gdf["geometry"].centroid

    buildings_gdf['volume']=buildings_gdf['area']*buildings_gdf['height']
    bld_dens=buildings_gdf['volume'].sum()/gdf_boundary.area
    bld_dens_city.append(bld_dens.values[0])

    # pop density
    if country=='Germany':
        dens=pd.read_excel('../outputs/density_geounits/summary_stats_' + city + '.xlsx',sheet_name='area_pop_sum_basic')
    else:
        dens=pd.read_excel('../outputs/density_geounits/summary_stats_' + city + '.xlsx',sheet_name='area_pop_sum')

    pop_dens=dens.loc[dens['index']=='density',0].values[0]
    pop_total=dens.loc[dens['index']=='population',0].values[0]
    area=dens.loc[dens['index']=='area',0].values[0]

    pop_dens_city.append(pop_dens)
    pop_city.append(pop_total)
    area_city.append(area)

    # distances
    data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')
    data=data.loc[:,['Res_geocode','HHNR','DistSubcenter_res', 'DistCenter_res']].drop_duplicates()

    dc=data['DistCenter_res'].mean()
    dsc=data['DistSubcenter_res'].mean()

    d2c.append(dc)
    d2sc.append(dsc)

    # land use
    year=ua_year[cities.index(city)]
    ver=ua_ver[cities.index(city)]

    if city=='Potsdam':
        fp='../../MSCA_data/UrbanAtlas/Berlin/Data/Berlin_UA2018_v013.gpkg'

    else: 
        fp='../../MSCA_data/UrbanAtlas/' + city + '/Data/' + city + '_UA' + year + '_' + ver +'.gpkg'
    LU=gpd.read_file(fp)
    LU.to_crs(crs0,inplace=True)

    LU_gdf=gpd.overlay(gdf_boundary,LU,how='intersection')

    if year =='2012': LU_gdf.rename(columns={'class_2012':'class_2018'},inplace=True)
    # create urban fabric (residential) and commercial and road land-uses
    LU_gdf['Class']='Other'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Discontinuous dense urban fabric (S.L. : 50% -  80%)','Discontinuous medium density urban fabric (S.L. : 30% - 50%)','Discontinuous low density urban fabric (S.L. : 10% - 30%)','Discontinuous very low density urban fabric (S.L. : < 10%)','Continuous urban fabric (S.L. : > 80%)']),'Class']='Urban_Fabric'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Industrial, commercial, public, military and private units','Construction sites','Airports','Port areas']),'Class']='Industrial_Commercial'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'Class']='Road'

    # make a non-road classification, to calculate by reverse the road area
    LU_gdf['RoadStatus']='NonRoad'
    LU_gdf.loc[LU_gdf['class_2018'].isin(['Fast transit roads and associated land','Other roads and associated land']),'RoadStatus']='Road'

    class_area=pd.DataFrame(LU_gdf.groupby('Class')['area'].sum()/LU_gdf['area'].sum()).reset_index()
    urb_fab.append(class_area.loc[class_area['Class']=='Urban_Fabric','area'].values[0])
    comm.append(class_area.loc[class_area['Class']=='Industrial_Commercial','area'].values[0])

    # connectivity

    poly = gdf_boundary.to_crs(4326).iloc[0].geometry

    graph_plz=ox.graph_from_polygon(poly,simplify=True,network_type=nw_type,retain_all=True)
    graph_proj=ox.project_graph(graph_plz)

    # get area
    graph_area=gdf_boundary.area.values[0]

    # get basic stats
    stats = ox.basic_stats(graph_proj,area=graph_area,clean_int_tol=10)

    # restrict to stats we are interested in
    stats1=dict((k, stats[k]) for k in metrics if k in stats)
    op=pd.Series(stats1)

    cf = '["cycleway"]'
    # this might throw an error if no graph is found within the polygon
    graph_bike = ox.graph_from_polygon(poly, custom_filter=cf,retain_all=True)
    if len(graph_bike.edges)>0:
        graph_proj2=ox.project_graph(graph_bike)
        stats = ox.basic_stats(graph_proj2,area=graph_area,clean_int_tol=10)
        stats2=dict((k, stats[k]) for k in metrics if k in stats)
        bike_lane_share=round(stats2['street_length_total']/stats1['street_length_total'],4)
    else:
        bike_lane_share=0
    #op=op.append(pd.Series({'bike_lane_share': bike_lane_share}))

    int_dens.append(op['clean_intersection_density_km'])
    street_len.append(op['street_length_avg'])
    bike_share.append(bike_lane_share)

    

    

In [4]:
c=pd.Series(cities)
c.apply(get_UF_avg)

df_summ=pd.DataFrame({'cities':c,'Population':pop_city,'Area':area_city,'Pop. density':pop_dens_city,'Built-up density':bld_dens_city,'Dist. to center':d2c,'Dist. to subcenter':d2sc,
                      'Instersec. density':int_dens,'Street length avg.':street_len,'Cycle lane share':bike_share,'Urban fabric area':urb_fab,'Commercial area':comm})
df_summ.to_csv('../outputs/summary_stats/summary_UF_all.csv',index=False)

Berlin
Dresden
Düsseldorf
Frankfurt am Main
Kassel


C:\Users\peter\AppData\Local\Temp\ipykernel_8652\2562996340.py:43: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')


Leipzig
Magdeburg
Potsdam
Clermont
Dijon
Lille
Lyon
Montpellier
Nantes
Nimes
Paris
Toulouse
Madrid


C:\Users\peter\AppData\Local\Temp\ipykernel_8652\2562996340.py:43: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../outputs/Combined/'+city+'_UF.csv')


Wien
